## Approximate Nearest Neighbor (ANN) Methods

In this practical, we would like to compare exhaustive search to ANN methods

  * Exhaustive Search, Product Quantization (PQ) and Vector Quantization

  * Will only count the time/memory of retrieval (as set up times, e.g. for indexing, only have to be done once)




# Prepare dataset and utility functions

* Download the SIFT1M dataset.
* Initialise some useful functions.


In [ ]:
import os
import time
import faiss
import shutil
import tarfile
import numpy as np
from contextlib import closing
import urllib.request as request

# first we download the Sift1M dataset
with closing(request.urlopen('ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz')) as r:
    with open('sift.tar.gz', 'wb') as f:
        shutil.copyfileobj(r, f)

# the download leaves us with a tar.gz file, we unzip it
tar = tarfile.open('sift.tar.gz', "r:gz")
tar.extractall()


In [ ]:
# function: read the fvecs file format of Sift1M dataset
def read_fvecs(fp):
    a = np.fromfile(fp, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy().view('float32')

In [ ]:
# function: compute memory usage 
def get_memory(index, inMB = True):
  faiss.write_index(index, './temp.index')
  filesize = os.path.getsize('./temp.index')
  os.remove('./temp.index')
  if inMB: return filesize / (1024*1024)
  else: return filesize


## Start benchmarking

This bring us to the three metrics of interest:

- **Speed**. How long does it take to find the 10 (or some other number) most similar vectors to the query? Hopefully less time than the brute-force algorithm needs; otherwise, what’s the point of indexing?

- **Memory usage**. How much RAM does the method require? More or less than the original vectors? Faiss supports searching only from RAM, as disk databases are orders of magnitude slower. Yes, even with SSDs.

- **Accuracy**. How well does the returned list of results match the brute-force search results? Accuracy can be evaluated by counting the number of queries for which the true nearest neighbor is returned first in the result list (a measure called 1-recall@1), or by measuring the average fraction of 10 nearest neighbors that are returned in the 10 first results (the “10-intersection” measure).



In [ ]:
# Exhaustive Search NN

def ESNN(xq, xb, k):
    D = xb.shape[1] # size of the vectors
    Nindex = faiss.IndexFlatL2(D) # build the index
    Nindex.add(xb)                # add vectors to the index, no processing is applied to the vectors
    start = time.time()
    Ndist, Ni = Nindex.search(xq, k)  # actual search, Ndist: distance, Ni: the index of the NN
    end = time.time()
    NMem = get_memory(Nindex)
    return [end-start, NMem, Ndist, Ni]

In [ ]:
# Index Product Quantization

def PQNN(xq, xb, k, m, nbits):
  # m: number of subvectors you'd like to split
  # nbits: number of bits per subquantizer, k_ = 2**nbits, i.e. number of centroids per sub-space
  D = xb.shape[1] # size of the vectors
  assert D % m == 0  # make sure it's divisible

  PQindex = faiss.IndexPQ(D, m, nbits) # build the index
  PQindex.train(xb)  # PQ training can take some time when using large nbits
  PQindex.add(xb)

  start = time.time()
  PQdist, PQi = PQindex.search(xq, k)
  end = time.time()
  PQMem = get_memory(PQindex)
  return [end-start, PQMem, PQdist, PQi]

In [ ]:
# Index IVFPQ
# To speed up our search time we can add another step, using an IVF index.

def IVFPQNN(xq, xb, k=5, nlist=2048, nbits=8, nprobe=2):
  # nlist: how many Voronoi cells
  # nbits: when using IVF+PQ, can use 8, higher nbits values are not supported
  # nprobe: number of centroids for searching

  D = xb.shape[1] # size of the vectors
  quantizer = faiss.IndexFlatL2(D)

  IVFPQindex = faiss.IndexIVFPQ(quantizer, D, nlist, m, nbits)
  IVFPQindex.train(xb)
  IVFPQindex.add(xb)
  IVFPQindex.nprobe = nprobe 

  start = time.time()
  IVFPQdist, IVFPQi = IVFPQindex.search(xq, k)
  end = time.time()
  IVFPQMem = get_memory(IVFPQindex)
  return [end-start, IVFPQMem, IVFPQdist, IVFPQi]

In [ ]:
def compareRecall(gt, pred):
  tp = 0
  for c, i in enumerate(gt): tp += int(i[0] in pred[c])
  recall = tp / gt.shape[0]
  return recall

In [ ]:
# data we will search through 
xb = read_fvecs('./sift/sift_base.fvecs')  # 1M samples
# also get some query vectors to search with
xq = read_fvecs('./sift/sift_query.fvecs')
# take just a hundred query (there are many in sift_query)
xq = xq[:100]
print(xb.shape)

In [ ]:
# Exhaustive Search NN
# for each query vector in xq, it looks for the k=5 nearest neighbors in the xb. 

ESpred = ESNN(xq=xq, xb=xb, k=5)
[EStime, ESmem, ESdist, ESi] = ESpred # Ndist: Euclidean distance, Ni: refers to the indices of the 5 NN.
numQ = xq.shape[0]
print('Exhaustive Search, searching {} vectors, time: {:.3f}s, memory: {:.2f}MB, recall: {:.2f}'.format(numQ, EStime, ESmem, 1.0))

In [ ]:
xb.shape[0] * 2*2/8/(1024*1024)

In [ ]:
xb.shape[0] * 2/(1024*1024)

In [ ]:
1.91 - 0.476837158203125

In [ ]:
1.92 - 1.433162841796875

In [ ]:
# Product Quantization NN
# m: number of subvectors you'd like to split
# nbits: number of bits per subquantizer

for m in [2, 4, 8, 16]:
  for nbits in [2, 4, 6, 8]:
    PQpred = PQNN(xq=xq, xb=xb, k=5, m=m, nbits=nbits)
    [PQtime, PQmem, PQdist, PQi] = PQpred
    numQ = xq.shape[0]
    recall = compareRecall(gt=ESi, pred=PQi)
    print('PQ NN, searching {} vectors, m: {}, nbits: {}, time: {:.3f}s, memory: {:.2f}MB, recall: {:.2f}'.format(numQ, m, nbits, PQtime, PQmem, recall))
  print('-'*60)


In [ ]:
PQpred = PQNN(xq=xq, xb=xb, k=5, m=4, nbits=16)
[PQtime, PQmem, PQdist, PQi] = PQpred
numQ = xq.shape[0]
recall = compareRecall(gt=ESi, pred=PQi)
print('PQ NN, searching {} vectors, m: {}, nbits: {}, time: {:.3f}s, memory: {:.2f}MB, recall: {:.2f}'.format(numQ, m, nbits, PQtime, PQmem, recall))

In [ ]:
# Product Quantization NN
# m: number of subvectors you'd like to split
# nbits: number of bits per subquantizer

for m in [2, 4, 8, 16]:
  for nbits in [2, 4, 6, 8]:
    PQpred = PQNN(xq=xq, xb=xb, k=5, m=m, nbits=nbits)
    [PQtime, PQmem, PQdist, PQi] = PQpred
    numQ = xq.shape[0]
    recall = compareRecall(gt=ESi, pred=PQi)
    print('PQ NN, searching {} vectors, m: {}, nbits: {}, time: {:.3f}s, memory: {:.2f}MB, recall: {:.2f}'.format(numQ, m, nbits, PQtime, PQmem, recall))
  print('-'*60)


In [ ]:
# Product Quantization NN
# m: number of subvectors you'd like to split
# nbits: number of bits per subquantizer

for m in [2, 4, 8, 16]:
  for nbits in [2, 4, 6, 8]:
    PQpred = PQNN(xq=xq, xb=xb, k=5, m=m, nbits=nbits)
    [PQtime, PQmem, PQdist, PQi] = PQpred
    numQ = xq.shape[0]
    recall = compareRecall(gt=ESi, pred=PQi)
    print('PQ NN, searching {} vectors, m: {}, nbits: {}, time: {:.3f}s, memory: {:.2f}MB, recall: {:.2f}'.format(numQ, m, nbits, PQtime, PQmem, recall))
  print('-'*60)


* Question: How can you further increase the recall ?

In [ ]:
# IVFPQ NN
# nlist:  # how many Voronoi cells
# nbits: # when using IVF+PQ, higher nbits values are not supported
# nprobe: # of cells you'd like to search.
nbits = 8
nlist = 2048
nprobe = 2
IVFPQpred = IVFPQNN(xq=xq, xb=xb, k=5, nlist=nlist, nbits=nbits, nprobe=nprobe)
numQ = xq.shape[0]
[IVFPQtime, IVFPQmem, IVFPQdist, IVFPQi] = IVFPQpred
recall = compareRecall(gt=ESi, pred=IVFPQi)
print('IVFPQ NN, searching {} vectors, nlist: {}, nbits: {}, nprobe: {}, time: {:.3f}s, memory: {:.2f}MB, recall: {:.2f}'.format(numQ, nlist, nbits, nprobe, IVFPQtime, IVFPQmem, recall))


- Quetion: try to increase nprobe, what happens ?

More detailed benchmark can be found at http://ann-benchmarks.com/faiss-ivf.html